# Child Speech Recognition - Google Colab Training

This notebook trains a Whisper-small model on children's speech data using Google Colab's free GPU.

**Before starting:**
1. Enable GPU: `Runtime` → `Change runtime type` → `T4 GPU`
2. Upload data to Google Drive (see instructions below)
3. Run cells sequentially

**Estimated time:**
- Setup: 5 minutes
- Quick test: 5-10 minutes  
- Full training: 3-6 hours

## 1. Setup Environment

In [ ]:
# Clone repository
!git clone https://github.com/ekshubina/childs_speech_recog_chall.git
%cd childs_speech_recog_chall

# Install dependencies (takes ~2 minutes)
!pip install -q -r requirements.txt

# Set PYTHONPATH so 'src' module is importable in subprocesses
import os
os.environ['PYTHONPATH'] = '/content/childs_speech_recog_chall'

In [ ]:
# Verify GPU and PyTorch installation
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be very slow.")
    print("Enable GPU: Runtime → Change runtime type → T4 GPU")

# Verify src module is importable
try:
    from src.utils.config import load_config
    print("✓ src module loaded successfully")
except ImportError as e:
    print(f"❌ ERROR: Cannot import src module: {e}")
    print("Re-run the setup cell above")

### Update Code from Git

Run this cell if you've pushed changes to GitHub and need to update the code in Colab.

In [ ]:
%cd /content/childs_speech_recog_chall
!git pull origin main

# Reinstall requirements if dependencies changed
!pip install -q -r requirements.txt

print("✓ Code updated from GitHub")

## 2. Load Data via Bore Tunnel

Transfer data from your Mac directly to Colab — no Google Drive needed.

**On your Mac, open two terminals:**

**Terminal 1** — serve your data directory:
```bash
cd /path/to/childs_speech_recog_chall/data
python3 -m http.server 8080
```

**Terminal 2** — expose it with bore:
```bash
bore local 8080 --to bore.pub
# Prints: listening at bore.pub:XXXXX  ← copy that port number
```

Then set `BORE_PORT` in the cell below and run it.

In [ ]:
import os
if os.path.exists('data'):
    !rm -r data

In [ ]:
# ← Set your bore port number here
BORE_PORT = "XXXXX"
BORE_URL = f"http://bore.pub:{BORE_PORT}"

!mkdir -p data

# Download zip files from your Mac
!wget -q --show-progress "{BORE_URL}/audio_0.zip" -O data/audio_0.zip
!wget -q --show-progress "{BORE_URL}/audio_1.zip" -O data/audio_1.zip
!wget -q --show-progress "{BORE_URL}/audio_2.zip" -O data/audio_2.zip
!wget -q --show-progress "{BORE_URL}/train_word_transcripts.jsonl" -O data/train_word_transcripts.jsonl

print("✓ Downloads complete")

In [ ]:
# Extract zip files
# Note: assumes each zip extracts to audio_0/, audio_1/, audio_2/ inside data/
# Run: !unzip -l data/audio_0.zip | head -5   to check structure if unsure
!unzip -q data/audio_0.zip -d data/
!unzip -q data/audio_1.zip -d data/
!unzip -q data/audio_2.zip -d data/

# Free up disk space
# !rm data/audio_*.zip

# Verify
from pathlib import Path

manifest_path = Path('data/train_word_transcripts.jsonl')
if manifest_path.exists():
    with open(manifest_path) as f:
        sample_count = sum(1 for _ in f)
    print(f"✓ Found training manifest with {sample_count:,} samples")
else:
    print("❌ ERROR: Training manifest not found!")

for audio_dir in ['audio_0', 'audio_1', 'audio_2']:
    audio_path = Path(f'data/{audio_dir}')
    if audio_path.exists():
        file_count = len(list(audio_path.glob('**/*.flac')))
        print(f"✓ Found data/{audio_dir}/ with {file_count:,} audio files")
    else:
        print(f"❌ ERROR: data/{audio_dir}/ not found! Check zip structure.")

## 3. Mount Google Drive

Checkpoints are saved locally during training (fast I/O), then copied to Drive afterwards.  
**Run this before the quick test and full training.**


In [ ]:
# Mount Google Drive and prepare config
# Checkpoints are written locally during training, then copied to Drive.
from google.colab import drive
drive.mount('/content/drive')

import os, yaml

# Local (fast) storage — used during training
LOCAL_CHECKPOINT_DIR = "/content/childs_speech_recog_chall/checkpoints"

# Google Drive destination
DRIVE_CHECKPOINT_DIR = "/content/drive/MyDrive/childs_speech_recog_chall/checkpoints"

os.makedirs(LOCAL_CHECKPOINT_DIR, exist_ok=True)
os.makedirs(DRIVE_CHECKPOINT_DIR, exist_ok=True)

# Write a patched config that redirects checkpoints to local storage.
# Both the quick test and full training use this same config.
RUN_NAME       = "baseline_whisper_small"
config_patched = "configs/baseline_whisper_small_local.yaml"

with open("configs/baseline_whisper_small.yaml") as f:
    cfg = yaml.safe_load(f)

local_output = f"{LOCAL_CHECKPOINT_DIR}/{RUN_NAME}"
cfg["training"]["output_dir"]  = local_output
cfg["training"]["logging_dir"] = f"{local_output}/runs"

with open(config_patched, "w") as f:
    yaml.dump(cfg, f, default_flow_style=False, allow_unicode=True)

print(f"✓ Drive mounted")
print(f"✓ Checkpoints during training → {local_output}")
print(f"✓ After training, copy to Drive → {DRIVE_CHECKPOINT_DIR}/{RUN_NAME}")


## 4. Quick Test (Recommended First Step)

Run a quick test on 100 samples to verify everything works before starting full training.  
After it finishes, run the **"Copy to Drive"** cell to confirm files appear on Drive.


In [ ]:
import shutil
from pathlib import Path

# Quick test — trains on 100 samples to verify the pipeline (~5-10 min)
!python scripts/train.py --config {config_patched} --debug

# Automatically copy to Drive when done
local_output = f"{LOCAL_CHECKPOINT_DIR}/{RUN_NAME}"
drive_output = f"{DRIVE_CHECKPOINT_DIR}/{RUN_NAME}"

if Path(local_output).exists():
    print("\nCopying checkpoints to Google Drive...")
    if Path(drive_output).exists():
        shutil.rmtree(drive_output)
    shutil.copytree(local_output, drive_output)
    print(f"✓ Saved to Drive: MyDrive/childs_speech_recog_chall/checkpoints/{RUN_NAME}/")
    print(f"   Files: {[p.name for p in sorted(Path(drive_output).iterdir())]}")
else:
    print("⚠️ Training did not produce checkpoints — check logs above for errors")


## 5. Full Training

⚠️ **This will take 3-6 hours** depending on GPU speed. Make sure:
- You have GPU enabled (Runtime → Change runtime type → T4 GPU)
- Your Colab session won't timeout (keep browser open or use Colab Pro)
- Run the **"Copy to Drive"** cell immediately after training finishes


In [ ]:
# Verify Drive setup is ready (variables set in the Drive setup cell after data extraction)
try:
    print(f"✓ LOCAL_CHECKPOINT_DIR : {LOCAL_CHECKPOINT_DIR}")
    print(f"✓ DRIVE_CHECKPOINT_DIR : {DRIVE_CHECKPOINT_DIR}")
    print(f"✓ config_patched       : {config_patched}")
except NameError:
    print("❌ Drive setup variables not found — re-run the Drive setup cell above (in Section 2)")


In [ ]:
import shutil
from pathlib import Path

# Full training — uses patched config created in cell 12
!python scripts/train.py --config {config_patched}

# Automatically copy to Drive when done
local_output = f"{LOCAL_CHECKPOINT_DIR}/{RUN_NAME}"
drive_output = f"{DRIVE_CHECKPOINT_DIR}/{RUN_NAME}"

if Path(local_output).exists():
    print("\nCopying checkpoints to Google Drive...")
    if Path(drive_output).exists():
        shutil.rmtree(drive_output)
    shutil.copytree(local_output, drive_output)
    print(f"✓ Saved to Drive: MyDrive/childs_speech_recog_chall/checkpoints/{RUN_NAME}/")
    print(f"   Files: {[p.name for p in sorted(Path(drive_output).iterdir())]}")
else:
    print("⚠️ Training did not produce checkpoints — check logs above for errors")


### Manual Copy to Drive (fallback only)

Checkpoints are copied automatically at the end of each training cell.  
Only run this if the automatic copy failed (e.g. training was interrupted).


In [ ]:
import shutil
from pathlib import Path

# ── Copy checkpoints to Google Drive ──────────────────────────────────────
# Run this cell right after training finishes.
# You can also re-run it independently at any time while the session is alive.

RUN_NAME     = "baseline_whisper_small"
local_output = f"{LOCAL_CHECKPOINT_DIR}/{RUN_NAME}"
drive_output = f"{DRIVE_CHECKPOINT_DIR}/{RUN_NAME}"

if not Path(local_output).exists():
    print(f"❌ Local checkpoint dir not found: {local_output}")
    print("   Training may not have completed yet — check the cell above.")
else:
    print(f"Copying {local_output}  →  {drive_output} ...")
    if Path(drive_output).exists():
        shutil.rmtree(drive_output)
    shutil.copytree(local_output, drive_output)
    print(f"✓ Done! Files on Drive:")

    for p in sorted(Path(drive_output).rglob("*"))[:30]:
        print(f"   {p.relative_to(drive_output)}")


## 6. Monitor Training with TensorBoard


In [ ]:
# Load TensorBoard — reads from local logs while training, Drive logs after copy
%load_ext tensorboard
%tensorboard --logdir {LOCAL_CHECKPOINT_DIR}/baseline_whisper_small/runs


## 7. Evaluation


In [ ]:
# Evaluate on validation set — uses model from Google Drive
MODEL_PATH = f"{DRIVE_CHECKPOINT_DIR}/baseline_whisper_small/final_model"

!python scripts/evaluate.py \
    --model-path {MODEL_PATH} \
    --val-manifest data/val_manifest.jsonl


## 8. Generate Predictions


In [ ]:
# Generate predictions on test set
!python scripts/predict.py \
    --model-path {MODEL_PATH} \
    --input-jsonl data/test_manifest.jsonl \
    --output-jsonl predictions.jsonl \
    --batch-size 16

# Copy predictions to Drive
!cp predictions.jsonl /content/drive/MyDrive/predictions.jsonl

print("✓ Predictions saved to Google Drive")

## 9. Download Results


In [ ]:
# Download predictions file to your local machine
from google.colab import files
files.download('predictions.jsonl')

## Troubleshooting

### No `childs_speech_recog_chall` folder on Google Drive
The folder is created and files are copied by the **"Copy to Drive"** cell (the one after training). If it didn't run (training crashed, session timed out, etc.):
1. Re-run the setup cell (cell 14) to remount Drive and recreate `LOCAL_CHECKPOINT_DIR` / `DRIVE_CHECKPOINT_DIR` variables
2. If local checkpoints still exist (`/content/childs_speech_recog_chall/checkpoints/`), re-run just the **"Copy to Drive"** cell
3. If local checkpoints are gone too (session restarted), resume from the last checkpoint already on Drive — see below

### Session Timeout During Training
If your Colab session disconnects **before the "Copy to Drive" cell completes**, the local checkpoints are lost.  
Resume from the last checkpoint that was already copied to Drive:
```python
RESUME_CKPT = "/content/drive/MyDrive/childs_speech_recog_chall/checkpoints/baseline_whisper_small/checkpoint-XXXX"
!python scripts/train.py \
    --config configs/baseline_whisper_small_local.yaml \
    --resume {RESUME_CKPT}
```
Then re-run the **"Copy to Drive"** cell.

### Bore connection drops / download stalls
Restart bore on your Mac and re-run the download cell with the new port.

### Wrong zip structure
Check what's inside your zip before extracting:
```python
!unzip -l data/audio_part_0.zip | head -10
```
If files are at root level (not inside `audio_0/`), extract to the specific folder:
```python
!unzip -q data/audio_part_0.zip -d data/audio_0/
```

### Out of Memory (OOM)
If you get CUDA OOM errors, reduce batch size:
```python
!sed -i 's/batch_size: 12/batch_size: 8/g' configs/baseline_whisper_small.yaml
```
Then restart runtime and try again.

### Slow Training
Verify GPU is enabled:
```python
import torch
print(torch.cuda.is_available())  # Should be True
```
